In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-22 21:33:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.5’

postgresql-42.2.16. 100%[===================>] 979.38K  4.74MB/s    in 0.2s    

2021-09-22 21:33:17 (4.74 MB/s) - ‘postgresql-42.2.16.jar.5’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         DE|      10133| RVOG49N0H1FB6|B004TACMZ8|     569741360|Bosch GMS120 Ortu...|    Home Improvement|          5|            0|          0|   N|                Y|               Super|Delivery took a l...| 2014-08-01|
|         DE|      19612| RNCMD6OLTP4HM|1846071224|     785505948|The Wheels On The...|               Bo

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
clean_review_df = df.select(["customer_id"])
clean_review_df.show()

+-----------+
|customer_id|
+-----------+
|      10133|
|      19612|
|      19612|
|      19677|
|      19999|
|      20014|
|      20152|
|      20152|
|      20152|
|      20152|
|      20152|
|      20152|
|      20152|
|      20152|
|      20376|
|      20376|
|      20402|
|      20426|
|      20426|
|      20426|
+-----------+
only showing top 20 rows



In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      38153|             3|
|    2094984|             1|
|    3006039|             1|
|    3121833|             1|
|    3391089|             8|
|    3470216|             2|
|    3529901|             1|
|    3744024|             1|
|    3768097|             1|
|    3804964|             1|
|    3908493|             1|
|    3949894|             1|
|    3975970|             1|
|    3980089|             1|
|    4614773|             2|
|    4672448|             2|
|    4816572|             1|
|    4876799|             1|
|    4991959|             1|
|    5011347|             1|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id']).drop_duplicates()
products_table_df = df.select(products_df["product_id"], df["product_title"])
products_table_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004TACMZ8|Bosch GMS120 Ortu...|
|1846071224|The Wheels On The...|
|0375851569|Dr. Seuss's  Begi...|
|B0060SVG54|   Zwei an einem Tag|
|B00EYQ6CVC|Dr. House - Die k...|
|B00MC1CCAG|     This is England|
|B005D7FKWI|Seiko Herren-Armb...|
|B007W9KQ9W|Battleship [Blu-ray]|
|B005F3DFXG|Planet Dinosaur [...|
|B00EYXGOYK|Thor  (+ BR) [3D ...|
|B00BRC6PYU|Cirque du Soleil:...|
|B00DZFMC4Y|Star Wars: The Cl...|
|B00F6MQUTW|Prometheus - Dunk...|
|B007IPJDWQ| The Thing [Blu-ray]|
|B009WIJW7S|Vielleicht lieber...|
|B008JR9DEA|Falling Skies - D...|
|B0041LG08U|Seiko Herren-Armb...|
|B00HFVE8QE|  Skateboard Party 2|
|B00QUY6MZM|        Interstellar|
|B00CA3C1N4|     Starlink (Full)|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
from pyspark.sql import Row
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
#merge_df = df.select(df["review_id"], df["customer_id"], products_df["product_id"], df["product_parent"], df["review_date"])
#review_id_table = merge_df.join(review_id_df, how='left')
#review_id_table = merge_df.na.drop(subset=["review_date", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RVOG49N0H1FB6|      10133|B004TACMZ8|     569741360| 2014-08-01|
| RNCMD6OLTP4HM|      19612|1846071224|     785505948| 2014-12-04|
| R4AUOBI8YC0R8|      19612|0375851569|     516548029| 2014-12-04|
|R1VSHIJ1RHIBTE|      19677|B0060SVG54|     302116447| 2015-07-16|
|R3JBLVALWSLCZD|      19999|B00EYQ6CVC|     368843515| 2014-02-08|
| RJ6GK77Y2NKCK|      20014|B00MC1CCAG|     663681053| 2014-11-11|
| R13W0E5EDX50J|      20152|B005D7FKWI|     182054491| 2013-10-04|
|R2L8UN6YSPY1CK|      20152|B007W9KQ9W|     251269761| 2013-10-04|
| RA2DFU68J832H|      20152|B005F3DFXG|     283218719| 2013-10-04|
|R191XKANDDV34N|      20152|B00EYXGOYK|     678332854| 2013-11-29|
| RD2NWWDF0S2B9|      20152|B00BRC6PYU|     715581077| 2014-05-21|
|R35SNJXI0Y9J1T|      20152|B00DZFMC4Y|     542832677| 2014-05

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RVOG49N0H1FB6|          5|            0|          0|   N|                Y|
| RNCMD6OLTP4HM|          5|            1|          1|   N|                Y|
| R4AUOBI8YC0R8|          5|            0|          0|   N|                Y|
|R1VSHIJ1RHIBTE|          5|            0|          0|   N|                Y|
|R3JBLVALWSLCZD|          5|            9|         14|   N|                Y|
| RJ6GK77Y2NKCK|          4|            6|          7|   N|                Y|
| R13W0E5EDX50J|          5|            0|          1|   N|                Y|
|R2L8UN6YSPY1CK|          3|            1|          3|   N|                Y|
| RA2DFU68J832H|          4|            1|          1|   N|                Y|
|R191XKANDDV34N|          4|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://josep-amazon-reviews.cn04rfli5rb1.us-east-1.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_df', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [14]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)